In [ ]:
# Add the path to the slack-pack/code/ folder in order to be able to import nlp module
import sys, os

NLP_PATH = '/'.join(os.path.abspath('.').split('/')[:-1]) + '/'
sys.path.append(NLP_PATH)

In [68]:
from collections import defaultdict
from pprint import pprint

from cassandra import ConsistencyLevel
from cassandra.cluster import Cluster

In [187]:
from nlp.text import extractor as xt

reload(xt)

<module 'nlp.text.extractor' from 'nlp/text/extractor.py'>

### Create a table with the same schema

In [102]:
q = """
CREATE TABLE IF NOT EXISTS boondoggle_messages (
    uuid text,
    message_text text,
    ts text,
    user text,
    team text,
    type text,
    channel text,
    PRIMARY KEY (uuid)
)
"""
session.execute(q)
print 'Created boondoggle_messages table!'

Created boondoggle_messages table!


In [185]:
# This will delete all entries in the table
# session.execute('TRUNCATE boondoggle_messages;')

### Boondoggle data to the Cassandra cluster...


In [188]:
insert_q = """
INSERT INTO boondoggle_messages (uuid, message_text, ts, user, team, type, channel)
VALUES ('%s', '%s', '%s', '%s', '%s', '%s', '%s')
"""

BASE_PATH = '/Users/will/Documents/Projects/local-slack-pack/data/Boondoggle_Sep_28_2016/{}/'

def process_channel(CHANNEL):
    print 'Processing {}: '.format(CHANNEL) 

    path_to_channel = BASE_PATH.format(CHANNEL.lower())
    counter = 0
    files_in_ch = !ls $path_to_channel

    for f in files_in_ch:
        filepath = path_to_channel + '/' + f

        print '|',

        ext = xt.JSONExtractor(file_name=filepath, logger='/Users/will/Documents/Projects/slack-pack/logs/extractor')
        m_stream = ext.get_messages()

        for m in m_stream:
            session.execute(insert_q % (m.id, m.text.encode('ascii', 'ignore').replace("'",''), m.timestamp,
                                       m.author, 'boondoggle', 'message', CHANNEL) )

            counter += 1

            if counter % 20 == 0:
                print '#',


In [1]:
# Get all channels to upload...
chnls = !ls /Users/will/Documents/Projects/local-slack-pack/data/Boondoggle_Sep_28_2016/
all_ch = [ c for c in chnls if '.json' not in c ]
all_ch

['bureaucrats',
 'eurotrash',
 'general',
 'legislators',
 'random',
 'sdcmihlss',
 'section5']

In [194]:
# FAILED_DICT = {}

# for chn in all_ch:
#     _f = process_channel(chn)
#     FAILED_DICT[chn] = _f
#     print ''

print 'This has been executed!'

This has been executed!


### Boondoggle messages report

In [199]:
_r = session.execute("SELECT * FROM boondoggle_messages")

channels_b = defaultdict(lambda: 0)

for i, r in enumerate(_r):
    channels_b[r.channel] += 1

print( 'Boondoggle has a total of #{} messages and #{} channels'.format(i, len(channels_b)) )

print( '\nChannel size report:')
for k, v in channels_b.iteritems():
    print '  {} --> size: {:3>}'.format(k,v)


Boondoggle has a total of #914 messages and #7 channels

Channel size report:
  sdcmihlss --> size: 79
  legislators --> size: 12
  bureaucrats --> size: 393
  random --> size: 12
  general --> size: 153
  eurotrash --> size: 19
  section5 --> size: 247
